In [2]:
key = ''


In [1]:
import os
import openai
from openai import OpenAI

In [3]:
openai.api_key=key
client=OpenAI(api_key=key)

In [8]:
def generate_gt(cve_description):
    """Generates the CWE mapping for a given CVE description using GPT-3.5."""
    prompt = f"""
    Analyze the following CVE description and map it to the appropriate CWE. 
    Provide a brief justification for your choice. 
    Ensure the last line of your response contains only the CWE ID.
    CVE Description: {cve_description}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "You are an expert in cybersecurity and CWE mappings."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=150
        )

        if response.choices and hasattr(response.choices[0].message, 'content'):
            content = response.choices[0].message.content.strip()
            result = content.split('\n')[-1]  # Get the last line
            print(f"Generated CWE ID: {result}")  # Debug print
            return result
        else:
            print("Error: Invalid response from OpenAI API")
            return None

    except openai.RateLimitError as e:
        print(f"Rate limit error: {e}")
        time.sleep(2 ** random.randint(1, 5))  # Exponential backoff
        return generate_gt(cve_description)  # Retry

    except Exception as e:
        print(f"Error: {e}")
        return None

In [21]:
import pandas as pd 

df = pd.read_csv('myRCM.tsv', sep='\t').sample(n=200)
df['gpt4o'] = df['Description'].apply(lambda x: generate_gt(x))
df.to_csv('gpt4MyData.tsv', sep='\t', index=False)


Generated CWE ID: CWE-200
Generated CWE ID: CWE-94
Generated CWE ID: These points fit well with CWE-400: Uncontrolled Resource Consumption ('Resource Exhaustion'), which
Generated CWE ID: CWE-121
Generated CWE ID: Such a vulnerability generally falls under the category of improper authorization or improper access control when an untrusted component (guest OS) is allowed to perform privileged operations that it should not have access to. This scenario aligns well with CWE-269: Improper Privilege Management. CWE-269 covers vulnerabilities where the software does not properly enforce privileges, allowing an attacker to leverage an incorrect assignment of the privileges assigned to user types and perform unauthorized actions
Generated CWE ID: CWE-79
Generated CWE ID: CWE-78
Generated CWE ID: The process of uploading arbitrary files without proper validation and restriction, particularly without authentication, broadens the potential for malicious activities, making the system vulnerable to

In [22]:
def compute_rcm_accuracy(fname, col):
    """Compute RCM accuracy by comparing predicted CWE with ground truth."""
    df = pd.read_csv(fname, sep='\t')
    correct = 0
    total = 0

    for idx, row in df.iterrows():
        pred = row[col].upper()
        #gt = row['GT']
        gt = row['CWE']


        # Validate if the prediction starts with 'CWE-'
        if pred.startswith('CWE-'):
            total += 1
            if pred == gt:
                correct += 1
        else:
            print(f'Invalid response at row {idx + 1}')

    # Calculate and return the accuracy
    accuracy = (correct / total) * 100 if total > 0 else 0
    print(f"Accuracy: {accuracy:.2f}%")
    return accuracy


In [23]:
# Compute the accuracy using the saved predictions
accuracy = compute_rcm_accuracy('gpt4MyData.tsv', 'gpt4o')
print(f"Final RCM Accuracy: {accuracy:.2f}%")

Invalid response at row 3
Invalid response at row 5
Invalid response at row 8
Invalid response at row 10
Invalid response at row 11
Invalid response at row 13
Invalid response at row 14
Invalid response at row 15
Invalid response at row 17
Invalid response at row 18
Invalid response at row 20
Invalid response at row 21
Invalid response at row 22
Invalid response at row 23
Invalid response at row 26
Invalid response at row 27
Invalid response at row 28
Invalid response at row 30
Invalid response at row 32
Invalid response at row 33
Invalid response at row 34
Invalid response at row 37
Invalid response at row 38
Invalid response at row 39
Invalid response at row 44
Invalid response at row 45
Invalid response at row 48
Invalid response at row 50
Invalid response at row 51
Invalid response at row 54
Invalid response at row 56
Invalid response at row 57
Invalid response at row 59
Invalid response at row 61
Invalid response at row 63
Invalid response at row 64
Invalid response at row 65
Inva